<a href="https://colab.research.google.com/github/HarshVardhanKumar/Chart.js/blob/master/musician.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from music21 import converter, instrument, note, chord
import glob
import numpy as np
from keras.utils import np_utils
from torch.utils.data import DataLoader, TensorDataset
import pickle

In [ ]:
# Acknowledgment: This part of Code taken from https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
vocabulary = []
i = 0
for tunes in glob.glob("/content/drive/My Drive/music/*.mid"):
    midi = converter.parse(tunes)
    notes = None
    parts = instrument.partitionByInstrument(midi)
    if parts: 
        notes = parts.parts[0].recurse()
    else: 
        notes = midi.flat.notes
    for element in notes:
        if isinstance(element, note.Note):
            vocabulary.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            vocabulary.append('.'.join(str(n) for n in element.normalOrder))

In [ ]:
with open('/content/drive/My Drive/notes', 'wb') as f:
  pickle.dump(vocabulary, f)

In [ ]:
with open('/content/drive/My Drive/notes', 'rb') as f:
  vocabulary = pickle.load(f)

inputs = []
outputs = []
seq_length = 250
unique_notes = set(vocabulary)
n_unique_notes = len(unique_notes)
pitchnames = sorted(unique_notes)

# Encoding the notes as integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

for i in range(0, len(vocabulary) - seq_length, 1):
        inputs.append([note_to_int[char] for char in vocabulary[i:i+seq_length]])
        outputs.append(note_to_int[vocabulary[i+seq_length]])

# the model has only single input for each sequence
inputs = np.reshape(outputs, (len(inputs), seq_lengths,1))

X_train = torch.from_numpy(np.asarray(inputs / float(n_unique_notes))).float()
Y_train = torch.from_numpy(np.asarray(np_utils.to_categorical(outputs))).float()

In [3]:
trainloader = DataLoader(TensorDataset(X_train, Y_train), batch_size = 32)

In [ ]:
# create the model
class LSTM_Music(nn.Module):
  def __init__(self, hidden_dim, vocab_size, input_size):
    super(LSTM_Music, self).__init__()
    #self.lstm = nn.LSTM(input_size = input_size, hidden_dim=hidden_dim, num_layers=2, dropout = 0.2, bidirectional=True)
    self.lstm = nn.LSTM(input_size, hidden_dim, dropout=0.2, num_layers=2, bidirectional=True)
    self.drop1 = nn.Dropout(p=0.2)
    self.gru = nn.GRU(2*hidden_dim, hidden_size = hidden_dim)
    self.drop2 = nn.Dropout(p=0.2)
    self.fc = nn.Linear(hidden_dim, vocab_size)
    self.out = nn.Softmax()
  
  def forward(self,x):
    x,_ = self.lstm(x)
    x=self.drop1(x)
    x,_ = self.gru(x)
    x = self.fc(x)
    x = self.out(x)
    return x[:,x.shape[1]-1,:]

hidden_dim = 500
input_size = X_train.shape[2]

if torch.cuda.is_available():
  device='cuda:0'
else:
  device = 'cpu'

model = LSTM_Music(input_size = input_size, hidden_dim = hidden_dim, vocab_size = n_unique_notes)
model.to(device)

In [ ]:
def train(lstm, dataloader, epochs=20):
  loss_fn = nn.BCELoss()
  min_cost = 99
  optimizer = optim.SGD(model.parameters(),lr=0.1)
  loss_fn.to(device)
  for e in range(epochs):
    lv = 0.0
    for input,output in dataloader:
      model.zero_grad();
      input = input.cuda()
      predictions = model(input)

      output = output.cuda()
      loss = loss_fn(predictions, output)
      loss.backward()
      optimizer.step()
      lv = lv+loss.item()
    print("Epoch "+str(e)+" loss = "+str(lv/len(dataloader)))
    if lv/len(dataloader) < min_cost :
      min_cost = lv/len(dataloader)
      torch.save(model.state_dict(), '/content/drive/My Drive/musician.pth')

In [ ]:
train(model, trainloader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0 loss = 0.04019640274956888
Epoch 1 loss = 0.040196401875253196
Epoch 2 loss = 0.04019638523616912
Epoch 3 loss = 0.040196381651348846
Epoch 4 loss = 0.04019639753328059
Epoch 5 loss = 0.040196388287140304
Epoch 6 loss = 0.0401963929632804
Epoch 7 loss = 0.04019638093734537
Epoch 8 loss = 0.040196389633258896
Epoch 9 loss = 0.04019638102836743


In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/musician.pth')